## GA4GH RNA Quantification API Analysis Example
This example uses the `rna_quantification_service` to retrieve data for a couple of simple analyses.

TODO: Bioconductor citation/acknowledgement

### Initialize client
In this step we create a client object which will be used to communicate with the server.  This example uses the GA4GH 1000 Genomes server.  It will also require that `scipy` and `rpy2` be installed to do the example calculations.

In [ ]:
import scipy.stats
from ga4gh.client import client
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
#c = client.HttpClient("http://1kgenomes.ga4gh.org")
c = client.HttpClient("http://localhost:8000")

### Search RNA Quantification Sets
Begin by retrieving a list of RNA quantification sets in the dataset.  RNA quantification sets are a way to associate a group of related RNA quantifications.  Note that the `dataset_id` is obtained as described in the `1kg_metadata_service` notebook.

In [ ]:
dataset_id = c.search_datasets().next().id
for rna_quant_set in c.search_rna_quantification_sets(dataset_id=dataset_id):
    rna_set_id = rna_quant_set.id
    print("id: {}".format(rna_quant_set.id))
    print("name: {}".format(rna_quant_set.name))

### Search RNA Quantifications
We can now list all of the RNA quantifications in the RNA quantification set.

In [ ]:
rna_quantification_ids = []
for rna_quant in c.search_rna_quantifications(rna_quantification_set_id=rna_set_id):
    print("({}): {}".format(rna_quant.id, rna_quant.name))
    rna_quantification_ids.append(rna_quant.id)

### Select RNA Quantifications
The RNA Quantification Set contains several RNA Quantifications.  Before we move on let's verify that the RNA Quantifications to be compared were processed with the same annotation.

In [ ]:
feature_sets = set()
for rna_quant_id in rna_quantification_ids:
    for feature_set_id in c.get_rna_quantification(
            rna_quantification_id=rna_quant_id).feature_set_ids:
        feature_sets.add(feature_set_id)
print("If == 1 we don't have to cull from the list --> {}".format(len(feature_sets)))

### Search Expression Levels
The feature level expression data for each RNA quantification is reported as a set of Expression Levels.  Here is an example.

In [ ]:
def getUnits(unitType):
    units = ["", "FPKM", "TPM"]
    return units[unitType]

counter = 0
expression_levels = []
for expression in c.search_expression_levels(rna_quantification_id=rna_quantification_ids[0]):
    if counter > 5:
        break
    counter += 1
    if expression.feature_id != "":
        expression_levels.append(expression)
    print("Expression Level: {}".format(expression.name))
    print(" id: {}".format(expression.id))
    print(" feature: {}".format(expression.feature_id))
    print(" expression: {} {}".format(expression.expression, getUnits(expression.units)))
    print(" read_count: {}".format(expression.raw_read_count))
    print(" confidence_interval: {} - {}\n".format(
            expression.conf_interval_low, expression.conf_interval_high))

### Finding Features in the other RNA Quantifications
We can also easily examine the expression level of these features in the other RNA Quantifications by specifying the feature_id in the `expressionLevelSearch`.

In [ ]:
feature_ids = [expression_levels[1].feature_id]
for rna_quantification_id in rna_quantification_ids[1:]:
    for expression in c.search_expression_levels(
            rna_quantification_id=rna_quantification_id, feature_ids=feature_ids):
        print("RNA Quantification: {}".format(rna_quantification_id))
        print("Expression Level: {}".format(expression.name))
        print(" id: {}".format(expression.id))
        print(" feature: {}".format(expression.feature_id))
        print(" expression: {} {}\n".format(expression.expression, getUnits(expression.units)))


### Spearman Correlation
Now that we can retrieve expression levels for the same feature for all the RNA Quantifications we can calculate the correlation between the Quantifications.  For this we will compute the Spearman Correlation using the `scipy` package.  Further, as this is just an example, we will just calculate the correlation using a small list of features instead of the entire quantification sets.

In [ ]:
def build_expression_dict(rna_quantification_id, max_features=50):
    counter = 0
    expression_dict = {}
    for expression in c.search_expression_levels(rna_quantification_id=rna_quantification_id):
        if counter > max_features:
            break
        counter += 1
        if expression.feature_id != "":
            expression_dict[expression.name] = expression.expression
    return expression_dict


expressions_dict_1 = build_expression_dict(rna_quantification_ids[0])
featureNames = set(expressions_dict_1.keys())
expressions_dict_2 = build_expression_dict(rna_quantification_ids[1])
featureNames = featureNames.intersection(set(expressions_dict_2.keys()))
sample_1 = []
sample_2 = []
featureNameList = list(featureNames) # preserve feature order
for feature_name in featureNameList:
    sample_1.append(expressions_dict_1[feature_name])
    sample_2.append(expressions_dict_2[feature_name])

scipy.stats.spearmanr(sample_1, sample_2)

### Differential Expression
Another common analysis is to look at the differential expression of features between datasets.  This is accomplished by using DESeq2 which is an R package.  This first step will use the `biocLite.R` script to install Bioconductor is needed or update it if already installed.

In [ ]:
utils = rpackages.importr('utils')
utils.chooseBioCmirror(ind=1)
robjects.r.source("https://bioconductor.org/biocLite.R")
robjects.r.biocLite()
robjects.r.biocLite("DESeq2")
robjects.r.biocLite(robjects.r.c("RColorBrewer", "pheatmap"))

### Creating the count matrix
DESeq2 operates on a count matrix in which each row contains a feature and each column corresponds to an RnaQuantification.  The values in the matrix are the counts for each feature for each Quantification.

In [ ]:
def build_expression_dict(rna_quantification_id, max_features=50):
    """
        We are going to rewrite this to return count as well as the quantification
        name so that we can build the required matrix.  Also, zero count features
        are going to be filtered out.
    """
    counter = 0
    expression_dict = {}
    quantification = c.get_rna_quantification(rna_quantification_id=rna_quantification_id)
    for expression in c.search_expression_levels(rna_quantification_id=rna_quantification_id):
        if counter > max_features:
            break
        counter += 1
        if expression.feature_id != "" and expression.raw_read_count > 0:
            expression_dict[expression.name] = expression.raw_read_count
    return quantification.name, expression_dict


rna_quantifications = []
name1, expressions_dict_1 = build_expression_dict(rna_quantification_ids[0])
rna_quantifications.append(name1)
featureNames = set(expressions_dict_1.keys())
name2, expressions_dict_2 = build_expression_dict(rna_quantification_ids[1])
rna_quantifications.append(name2)
featureNames = featureNames.intersection(set(expressions_dict_2.keys()))
name3, expressions_dict_3 = build_expression_dict(rna_quantification_ids[2])
rna_quantifications.append(name3)
featureNames = featureNames.intersection(set(expressions_dict_3.keys()))
name4, expressions_dict_4 = build_expression_dict(rna_quantification_ids[3])
rna_quantifications.append(name4)
featureNames = featureNames.intersection(set(expressions_dict_4.keys()))
name5, expressions_dict_5 = build_expression_dict(rna_quantification_ids[4])
rna_quantifications.append(name5)
featureNames = featureNames.intersection(set(expressions_dict_5.keys()))
sample_1 = []
sample_2 = []
sample_3 = []
sample_4 = []
sample_5 = []
conditions = ["sample_1", "sample_2", "sample_3", "sample_4", "sample_5"]
featureNameList = list(featureNames) # preserve feature order
for feature_name in featureNameList:
    sample_1.append(expressions_dict_1[feature_name])
    sample_2.append(expressions_dict_2[feature_name])
    sample_3.append(expressions_dict_3[feature_name])
    sample_4.append(expressions_dict_4[feature_name])
    sample_5.append(expressions_dict_5[feature_name])
count_m = None
count_m = robjects.r.cbind(robjects.IntVector(sample_1), robjects.IntVector(sample_2))
count_m = robjects.r.cbind(count_m, robjects.IntVector(sample_3))
count_m = robjects.r.cbind(count_m, robjects.IntVector(sample_4))
count_m = robjects.r.cbind(count_m, robjects.IntVector(sample_5))
count_m.rownames = robjects.StrVector(featureNameList)
count_m.colnames = robjects.StrVector(rna_quantifications)
coldata = robjects.r.cbind(robjects.StrVector(rna_quantifications),
                           robjects.StrVector(conditions))
coldata.colnames = robjects.StrVector(["library", "sample"])

### Ready for DESeq
Now that the count matrix and metadata tables are done the next step is to create the `DESeqDataSet` and then run the pipeline.

In [ ]:
robjects.r.library("DESeq2")
design = robjects.Formula("~ library")
ddsMat = robjects.r.DESeqDataSetFromMatrix(countData=count_m, colData=coldata, design=design)
deseq_result = robjects.r.DESeq(ddsMat)
# robjects.r.results shows the last result
print(robjects.r.results(deseq_result))
# can also look any of the results by specifying contrast:
print(robjects.r.results(deseq_result, contrast=robjects.r.c("library", name1, name2)))

### Clustering
It is often interesting to see how samples cluster with respect to gene expression.

In [ ]:
robjects.r.X11()
robjects.r.library("pheatmap")
rld = robjects.r.rlog(deseq_result, blind=False)
print(robjects.r.head(robjects.r.assay(rld), 3))
robjects.r.pheatmap(robjects.r.assay(rld))